Data visualization and transformation is an important part of every model. Now that we have our dataset downloaded, let's learn more about audio data visualization and transforming this dataset.

TorchAudio has many transformation functions for audio manipulation and feature extractions. However, in this module, your focus is on the following concepts and transforms: `Spectrogram`, `MelSpectrogram`, `Waveform`, and `MFCC`. After you understand these concepts, you'll create your spectrogram images of the yes/no dataset to be used in the computer vision model.

- **Spectrogram**: Create a spectrogram from a waveform.
- **MFCC**: Create the Mel-frequency cepstrum coefficients from a waveform.
- **MelSpectrogram**: Create Mel spectrograms from a waveform by using the `STFT` function in PyTorch.

## Load the dataset folders into a data loader

Import the packages, and create a `load_audio_files` function to load audio files from a specified path into a dataset.

In [ ]:
import os
import torch
import torchaudio
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

First, we'll go through the audio file that we downloaded in our local directory by filtering out the ones that are the `yes` and `no` commands under the _nohash_ path. Then we'll load the files into the `torchaudio` data object. This will make it easy to extract the attributes of the audio (for example, the waveform and sample rate).

In [ ]:
default_dir = os.getcwd()
folder = 'data'

def load_audio_files(path: str, label:str):

    dataset = []
    walker = sorted(str(p) for p in Path(path).glob(f'*.wav'))

    for i, file_path in enumerate(walker):
        path, filename = os.path.split(file_path)
        speaker, _ = os.path.splitext(filename)
        speaker_id, utterance_number = speaker.split("_nohash_")
        utterance_number = int(utterance_number)
    
        # Load audio
        waveform, sample_rate = torchaudio.load(file_path)
        dataset.append([waveform, sample_rate, label, speaker_id, utterance_number])
        
    return dataset

Call the `load_audio_files` function to load the contents from each of the audio class files, as well as their metadata.  

In [ ]:
trainset_speechcommands_yes = load_audio_files(f'./{folder}/SpeechCommands/speech_commands_v0.02/yes', 'yes')
trainset_speechcommands_no = load_audio_files(f'./{folder}/SpeechCommands/speech_commands_v0.02/no', 'no')

In [ ]:
print(f'Length of yes dataset: {len(trainset_speechcommands_yes)}')
print(f'Length of no dataset: {len(trainset_speechcommands_no)}')

Now load the dataset into a data loader for both `yes` and `no` training sample sets. `DataLoader` sets the number of batches you want to iterate to load the dataset through your network, to train the model. We'll set the batch size to 1, because we want to load the entire batch in one iteration.

In [ ]:
trainloader_yes = torch.utils.data.DataLoader(trainset_speechcommands_yes, batch_size=1,
                                            shuffle=True, num_workers=0)

In [ ]:
trainloader_no = torch.utils.data.DataLoader(trainset_speechcommands_no, batch_size=1,
                                            shuffle=True, num_workers=0)

To see how the data looks, we'll grab the waveform and sample rate from each class, and print out a sample of the dataset.

- The **waveform** value is in a Tensor with a float datatype.
- The **sample rate** value is 16000 in the format the audio signal was captured.
- The **label** value is the command classification of the word uttered in the audio, `yes` or `no`.
- The **ID** is a unique identifier of the audio file.

In [ ]:
yes_waveform = trainset_speechcommands_yes[0][0]
yes_sample_rate = trainset_speechcommands_yes[0][1]
print(f'Yes Waveform: {yes_waveform}')
print(f'Yes Sample Rate: {yes_sample_rate}')
print(f'Yes Label: {trainset_speechcommands_yes[0][2]}')
print(f'Yes ID: {trainset_speechcommands_yes[0][3]} \n')

no_waveform = trainset_speechcommands_no[0][0]
no_sample_rate = trainset_speechcommands_no[0][1]
print(f'No Waveform: {no_waveform}')
print(f'No Sample Rate: {no_sample_rate}')
print(f'No Label: {trainset_speechcommands_no[0][2]}')
print(f'No ID: {trainset_speechcommands_no[0][3]}')

## Transform and visualize

Our data is ready! Let's break down some of the audio transforms and the visualization to better understand what they are, and what they tell us about the data.

### Waveform

The waveform is generated by taking the sample rate and frequency, and representing the signal visually. This signal can be represented as a `waveform`, which is the `signal` representation over time, in a graphical format. The audio can be recorded in different `channels`. For example, stereo recordings have two channels: right and left.

Here's how to use the `resample` transform to reduce the size of the waveform, and then graph the data to visualize the new waveform shape.

In [ ]:
def show_waveform(waveform, sample_rate, label):
    print("Waveform: {}\nSample rate: {}\nLabels: {} \n".format(waveform, sample_rate, label))
    new_sample_rate = sample_rate/10
   
    # Resample applies to a single channel, we resample first channel here
    channel = 0
    waveform_transformed = torchaudio.transforms.Resample(sample_rate, new_sample_rate)(waveform[channel,:].view(1,-1))

    print("Shape of transformed waveform: {}\nSample rate: {}".format(waveform_transformed.size(), new_sample_rate))

    plt.figure()
    plt.plot(waveform_transformed[0,:].numpy())

The displayed results show how the sample rate is transformed from 16000 to 1600.

In [ ]:
show_waveform(yes_waveform, yes_sample_rate, 'yes')

In [ ]:
show_waveform(no_waveform, no_sample_rate, 'no')

## Spectrogram

So, what is a spectrogram? A spectrogram maps the frequency to time of an audio file, and it allows you to visualize audio data by frequency. It's in an image format. This image is what we'll use for our computer vision classification on the audio files. You can view the spectrogram image in grayscale, or in Red Green Blue (RGB) color format.

Every spectrogram image helps show the different features the sound signal produces in a color pattern. The convolutional neural network (CNN) treats the color patterns in the image as features for training the model to classify the audio.

Let's use the PyTorch `torchaudio.transforms` function to transform the waveform to a spectrogram image format.


In [ ]:
def show_spectrogram(waveform_classA, waveform_classB):
    yes_spectrogram = torchaudio.transforms.Spectrogram()(waveform_classA)
    print("\nShape of yes spectrogram: {}".format(yes_spectrogram.size()))
    
    no_spectrogram = torchaudio.transforms.Spectrogram()(waveform_classB)
    print("Shape of no spectrogram: {}".format(no_spectrogram.size()))

    plt.figure()
    plt.subplot(1, 2, 1)
    plt.title("Features of {}".format('no'))
    plt.imshow(yes_spectrogram.log2()[0,:,:].numpy(), cmap='viridis')
    
    plt.subplot(1, 2, 2)
    plt.title("Features of {}".format('yes'))
    plt.imshow(no_spectrogram.log2()[0,:,:].numpy(), cmap='viridis')  

We'll use the waveform for the `yes` command to display the spectrogram images dimensions and color pattern in an RGB chart. We'll also compare the feature difference between the `yes` and `no` audio commands.

- The **y-axis** is the frequency of the audio.
- The **x-axis** is the time of the audio.
- The intensity of the image shows the amplitude or pitch of the audio. In the following spectrogram images, the high concentrate of the yellow color illustrates the amplitude of the audio.

In [ ]:
show_spectrogram(yes_waveform, no_waveform)

## Mel spectrogram

Mel spectrogram is also a frequency to time, but the frequency is converted to the Mel scale. The Mel scale takes the frequency and changes it, based on the perception of the sound of the scale or melody. This transforms the frequency within to the Mel scale, and then creates the spectrogram image.

In [ ]:
def show_melspectrogram(waveform,sample_rate):
    mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate)(waveform)
    print("Shape of spectrogram: {}".format(mel_spectrogram.size()))

    plt.figure()
    plt.imshow(mel_spectrogram.log2()[0,:,:].numpy(), cmap='viridis')

In [ ]:
show_melspectrogram(yes_waveform, yes_sample_rate)

## Mel-frequency cepstral coefficients (MFCC)

A simplified explanation of what the MFCC does is that it takes our frequency, applies transforms, and the result is the amplitudes of the spectrum created from the frequency. Let's take a look at what this looks like.

In [ ]:
def show_mfcc(waveform,sample_rate):
    mfcc_spectrogram = torchaudio.transforms.MFCC(sample_rate= sample_rate)(waveform)
    print("Shape of spectrogram: {}".format(mfcc_spectrogram.size()))

    plt.figure()
    fig1 = plt.gcf()
    plt.imshow(mfcc_spectrogram.log2()[0,:,:].numpy(), cmap='viridis')
    
    plt.figure()
    plt.plot(mfcc_spectrogram.log2()[0,:,:].numpy())
    plt.draw()

In [ ]:
show_mfcc(no_waveform,  no_sample_rate)

## Create an image from a spectrogram

At this point, you have a better understanding of your audio data, and different transformations you can use on it. Now, let's create the images we will use for classification. 

The following are two different functions to create the spectrogram image or the MFCC images for classification. You'll use the spectrogram images to train our model. Feel free to play around with MFCC classification on your own, if you want to.

In [ ]:
def create_spectrogram_images(trainloader, label_dir):
    #make directory
    directory = f'./{folder}/spectrograms/{label_dir}/'
    if(os.path.isdir(directory)):
        print("Data exists for", label_dir)
    else:
        os.makedirs(directory, mode=0o777, exist_ok=True)
        
        for i, data in enumerate(trainloader):

            waveform = data[0]
            sample_rate = data[1][0]
            label = data[2]
            ID = data[3]

            # create transformed waveforms
            spectrogram_tensor = torchaudio.transforms.Spectrogram()(waveform)     
            
            fig = plt.figure()
            plt.imsave(f'./{folder}/spectrograms/{label_dir}/spec_img{i}.png', spectrogram_tensor[0].log2()[0,:,:].numpy(), cmap='viridis')


Here's the `define` function to create the MFCC images.

In [ ]:
def create_mfcc_images(trainloader, label_dir):
    #make directory
    os.makedirs(f'./{folder}/mfcc_spectrograms/{label_dir}/', mode=0o777, exist_ok=True)
    
    for i, data in enumerate(trainloader):

        waveform = data[0]
        sample_rate = data[1][0]
        label = data[2]
        ID = data[3]
        
        mfcc_spectrogram = torchaudio.transforms.MFCC(sample_rate= sample_rate)(waveform)

        plt.figure()
        fig1 = plt.gcf()
        plt.imshow(mfcc_spectrogram[0].log2()[0,:,:].numpy(), cmap='viridis')
        plt.draw()
        fig1.savefig(f'./{folder}/mfcc_spectrograms/{label_dir}/spec_img{i}.png', dpi=100)
 
        #spectorgram_train.append([spectrogram_tensor, label, sample_rate, ID])

Create the spectrogram images that you'll use for the audio classification.

In [ ]:
create_spectrogram_images(trainloader_yes, 'yes')
create_spectrogram_images(trainloader_no, 'no')

We now have our audio as spectrogram images and are ready to build the model!